In [6]:
import requests
import json
import math
import time

# --- 설정 ---
API_KEY = "0fedb3690ed2dbeb688fb9f77f9b7fdc"
OUTPUT_FILENAME = "kobis_people_list.json"
# ------------

BASE_URL = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"
ITEMS_PER_PAGE = 100

def create_full_people_list():
    """KOBIS에 등록된 모든 영화인의 기본 목록을 생성하여 파일로 저장합니다."""
    print("전체 영화인 목록 생성을 시작합니다. 이 작업은 몇 분 정도 소요될 수 있습니다.")
    
    try:
        # 1. 첫 페이지를 요청하여 전체 인원 수 확인
        params = {'key': API_KEY, 'itemPerPage': str(ITEMS_PER_PAGE), 'curPage': '1'}
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        
        search_result = response.json()
        
        if 'faultInfo' in search_result:
            print(f"[오류] API 요청 실패: {search_result['faultInfo']['message']}")
            return

        total_count = search_result['peopleListResult']['totCnt']
        if total_count == 0:
            print("조회된 영화인이 없습니다.")
            return
            
        total_pages = math.ceil(total_count / ITEMS_PER_PAGE)
        print(f"총 {total_count}명의 영화인 목록을 발견했습니다. (전체 {total_pages} 페이지)")

        # 2. 모든 페이지를 순회하며 데이터 수집
        all_people_list = []
        for page in range(1, total_pages + 1):
            params['curPage'] = str(page)
            response = requests.get(BASE_URL, params=params)
            
            page_data = response.json()
            people_on_page = page_data['peopleListResult']['peopleList']
            all_people_list.extend(people_on_page)
            
            print(f"{page}/{total_pages} 페이지 목록 수집 중... ({len(all_people_list)}명)")
            time.sleep(0.1)

        # 3. 전체 목록을 파일에 저장
        with open(OUTPUT_FILENAME, 'w', encoding='utf-8') as f:
            json.dump(all_people_list, f, ensure_ascii=False, indent=4)

        print("\n--- 목록 생성 완료 ---")
        print(f"'{OUTPUT_FILENAME}' 파일에 {len(all_people_list)}명의 전체 목록을 성공적으로 저장했습니다.")
        print("이제 상세 정보 수집용 '스마트 코드'를 실행할 수 있습니다.")

    except Exception as e:
        print(f"작업 중 오류가 발생했습니다: {e}")

if __name__ == "__main__":
    create_full_people_list()

전체 영화인 목록 생성을 시작합니다. 이 작업은 몇 분 정도 소요될 수 있습니다.
총 172770명의 영화인 목록을 발견했습니다. (전체 1728 페이지)
1/1728 페이지 목록 수집 중... (100명)
2/1728 페이지 목록 수집 중... (200명)
3/1728 페이지 목록 수집 중... (300명)
4/1728 페이지 목록 수집 중... (400명)
5/1728 페이지 목록 수집 중... (500명)
6/1728 페이지 목록 수집 중... (600명)
7/1728 페이지 목록 수집 중... (700명)
8/1728 페이지 목록 수집 중... (800명)
9/1728 페이지 목록 수집 중... (900명)
10/1728 페이지 목록 수집 중... (1000명)
11/1728 페이지 목록 수집 중... (1100명)
12/1728 페이지 목록 수집 중... (1200명)
13/1728 페이지 목록 수집 중... (1300명)
14/1728 페이지 목록 수집 중... (1400명)
15/1728 페이지 목록 수집 중... (1500명)
16/1728 페이지 목록 수집 중... (1600명)
17/1728 페이지 목록 수집 중... (1700명)
18/1728 페이지 목록 수집 중... (1800명)
19/1728 페이지 목록 수집 중... (1900명)
20/1728 페이지 목록 수집 중... (2000명)
21/1728 페이지 목록 수집 중... (2100명)
22/1728 페이지 목록 수집 중... (2200명)
23/1728 페이지 목록 수집 중... (2300명)
24/1728 페이지 목록 수집 중... (2400명)
25/1728 페이지 목록 수집 중... (2500명)
26/1728 페이지 목록 수집 중... (2600명)
27/1728 페이지 목록 수집 중... (2700명)
28/1728 페이지 목록 수집 중... (2800명)
29/1728 페이지 목록 수집 중... (2900명)
30/1728 페이지 목록 수집 중... (3000명)
3

In [4]:
import requests
import json
import csv
import math
import time

# --- 설정 ---
API_KEY = "d29200c5aa880538982113d33c889935" # 새로 제공된 API 키
# ------------

BASE_URL = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json"
ITEMS_PER_PAGE = 100 # 한 페이지에 100명씩 조회
CHUNK_SIZE = 10000   # CSV 파일 하나에 저장할 인원 수

def save_chunk_to_csv(data, file_number):
    """리스트 데이터를 CSV 파일로 저장하는 함수"""
    filename = f"kobis_people_list_{file_number}.csv"
    try:
        with open(filename, 'w', newline='', encoding='utf-8-sig') as f:
            # CSV 파일의 헤더(첫 번째 줄) 정의
            header = ['peopleCd', 'peopleNm', 'peopleNmEn', 'repRoleNm', 'filmoNames']
            writer = csv.DictWriter(f, fieldnames=header)
            writer.writeheader()
            writer.writerows(data)
        print(f"\n✅ --- {len(data)}명의 데이터를 '{filename}' 파일로 저장했습니다. --- ✅\n")
    except Exception as e:
        print(f"[오류] '{filename}' 파일 저장 중 문제가 발생했습니다: {e}")

def create_full_people_list_in_chunks():
    """모든 영화인 목록을 10,000명 단위의 CSV 파일로 나눠서 저장합니다."""
    print("전체 영화인 목록 생성을 시작합니다. 10,000명 단위로 CSV 파일이 생성됩니다.")
    
    try:
        # 1. 첫 페이지를 요청하여 전체 인원 수 확인
        params = {'key': API_KEY, 'itemPerPage': str(ITEMS_PER_PAGE), 'curPage': '1'}
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        
        search_result = response.json()
        
        if 'faultInfo' in search_result:
            print(f"[오류] API 요청 실패: {search_result['faultInfo']['message']}")
            return

        total_count = search_result['peopleListResult']['totCnt']
        if total_count == 0:
            print("조회된 영화인이 없습니다.")
            return
            
        total_pages = math.ceil(total_count / ITEMS_PER_PAGE)
        print(f"총 {total_count}명의 영화인 목록을 발견했습니다. (전체 {total_pages} 페이지)")

        # 2. 모든 페이지를 순회하며 데이터 수집 및 저장
        chunk_data = [] # 10,000명을 임시로 담을 리스트
        file_number = 1   # CSV 파일 번호

        for page in range(1, total_pages + 1):
            params['curPage'] = str(page)
            response = requests.get(BASE_URL, params=params)
            
            page_data = response.json()
            people_on_page = page_data['peopleListResult']['peopleList']
            
            chunk_data.extend(people_on_page)
            
            total_collected = ((file_number - 1) * CHUNK_SIZE) + len(chunk_data)
            print(f"{page}/{total_pages} 페이지 목록 수집 중... (현재까지 총 {total_collected}명)")
            
            # 현재 청크가 10,000명이 되면 파일로 저장
            if len(chunk_data) >= CHUNK_SIZE:
                save_chunk_to_csv(chunk_data, file_number)
                # 다음 파일을 위해 리스트를 비우고 파일 번호를 1 증가시킴
                chunk_data = []
                file_number += 1
            
            time.sleep(0.1)

        # 3. 마지막으로 남은 데이터 저장
        # 모든 페이지를 다 돌고 10,000명이 채 안된 나머지 데이터를 저장합니다.
        if chunk_data:
            save_chunk_to_csv(chunk_data, file_number)

        print("\n--- 모든 작업 완료 ---")
        print("전체 영화인 목록을 여러 개의 CSV 파일로 성공적으로 저장했습니다.")

    except Exception as e:
        print(f"작업 중 오류가 발생했습니다: {e}")

if __name__ == "__main__":
    create_full_people_list_in_chunks()

전체 영화인 목록 생성을 시작합니다. 10,000명 단위로 CSV 파일이 생성됩니다.
총 172772명의 영화인 목록을 발견했습니다. (전체 1728 페이지)
1/1728 페이지 목록 수집 중... (현재까지 총 100명)
2/1728 페이지 목록 수집 중... (현재까지 총 200명)
3/1728 페이지 목록 수집 중... (현재까지 총 300명)
4/1728 페이지 목록 수집 중... (현재까지 총 400명)
5/1728 페이지 목록 수집 중... (현재까지 총 500명)
6/1728 페이지 목록 수집 중... (현재까지 총 600명)
7/1728 페이지 목록 수집 중... (현재까지 총 700명)
8/1728 페이지 목록 수집 중... (현재까지 총 800명)
9/1728 페이지 목록 수집 중... (현재까지 총 900명)
10/1728 페이지 목록 수집 중... (현재까지 총 1000명)
11/1728 페이지 목록 수집 중... (현재까지 총 1100명)
12/1728 페이지 목록 수집 중... (현재까지 총 1200명)
13/1728 페이지 목록 수집 중... (현재까지 총 1300명)
14/1728 페이지 목록 수집 중... (현재까지 총 1400명)
15/1728 페이지 목록 수집 중... (현재까지 총 1500명)
16/1728 페이지 목록 수집 중... (현재까지 총 1600명)
17/1728 페이지 목록 수집 중... (현재까지 총 1700명)
18/1728 페이지 목록 수집 중... (현재까지 총 1800명)
19/1728 페이지 목록 수집 중... (현재까지 총 1900명)
20/1728 페이지 목록 수집 중... (현재까지 총 2000명)
21/1728 페이지 목록 수집 중... (현재까지 총 2100명)
22/1728 페이지 목록 수집 중... (현재까지 총 2200명)
23/1728 페이지 목록 수집 중... (현재까지 총 2300명)
24/1728 페이지 목록 수집 중... (현재까지 총 2400명)
25/1728 페이지 목록 수집

In [ ]:
import requests
import json
import csv
import glob
import time
import os
import re

# --- 설정 ---
API_KEY =     "4dfdbf4abdc4df1cadf6f1d48f79ef80",
 # 지정하신 API 키
MAX_TO_COLLECT = 3000 # 이번에 수집할 목표 개수

# 상세정보를 저장할 폴더 경로
OUTPUT_FOLDER_PATH = r"C:\Users\yusse\OneDrive\바탕 화면\BAF-25-2-Marketing\배우추출_api\배우목록데이터"
# ------------
# "4dfdbf4abdc4df1cadf6f1d48f79ef80"
# "05fe695987e6b27c95631ba5bee9c601"
# "95fb114c7adc964e2a363a506cb32345"
# "d29200c5aa880538982113d33c889935"
# "3d9e9fb81650387f1b8f00b30e78b0f0"
# "0fedb3690ed2dbeb688fb9f77f9b7fdc"

DETAIL_BASE_URL = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"

def save_single_person_to_csv(person_data, full_path):
    """한 명의 상세정보를 CSV 파일로 저장하는 함수"""
    try:
        header = ['peopleCd', 'peopleNm', 'peopleNmEn', 'sex', 'repRoleNm', 'filmos', 'homepages']
        with open(full_path, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=header)
            writer.writeheader()
            person_data['filmos'] = json.dumps(person_data.get('filmos', []), ensure_ascii=False)
            person_data['homepages'] = json.dumps(person_data.get('homepages', []), ensure_ascii=False)
            writer.writerow(person_data)
        print(f"  -> ✅ 저장 완료: {os.path.basename(full_path)}")
    except Exception as e:
        print(f"  -> ❌ 저장 실패: {os.path.basename(full_path)} ({e})")

def collect_specific_amount():
    """정해진 개수만큼만 상세 정보를 수집합니다."""
    
    os.makedirs(OUTPUT_FOLDER_PATH, exist_ok=True)
    csv_files = glob.glob("kobis_people_list_*.csv")
    
    if not csv_files:
        print("[오류] 'kobis_people_list_*.csv' 파일을 찾을 수 없습니다.")
        return
        
    print(f"총 {len(csv_files)}개의 목록 CSV 파일을 발견했습니다. {MAX_TO_COLLECT}개 추가 수집을 시작합니다.")
    print("-" * 50)
    
    collected_count = 0 # 이번 작업에서 새로 수집한 개수 카운터

    for filename in sorted(csv_files):
        print(f"\n📄 '{filename}' 파일 작업을 시작합니다...")
        
        try:
            with open(filename, 'r', encoding='utf-8-sig') as f:
                reader = csv.DictReader(f)
                
                for row in reader:
                    people_cd = row.get('peopleCd')
                    people_nm = row.get('peopleNm', '이름없음')
                    if not people_cd: continue

                    safe_people_nm = re.sub(r'[\\/*?:"<>|]', "", people_nm)
                    output_filename = f"{people_cd}_{safe_people_nm}.csv"
                    output_full_path = os.path.join(OUTPUT_FOLDER_PATH, output_filename)

                    # 이미 파일이 있으면 건너뛰기 (9,000개를 건너뛰는 로직)
                    if os.path.exists(output_full_path):
                        continue

                    # --- 목표치 도달 시 작업 종료 ---
                    if collected_count >= MAX_TO_COLLECT:
                        print(f"\n[작업 완료] 목표치인 {MAX_TO_COLLECT}개를 모두 수집하여 작업을 중단합니다.")
                        return
                    # --------------------------------

                    print(f"\n[수집 현황: {collected_count + 1}/{MAX_TO_COLLECT}] {people_nm} ({people_cd}) 정보 수집...")
                    
                    params = {'key': API_KEY, 'peopleCd': people_cd}
                    try:
                        response = requests.get(DETAIL_BASE_URL, params=params)
                        result_json = response.json()

                        if 'faultInfo' in result_json:
                            error_msg = result_json['faultInfo']['message']
                            print(f"  -> API 오류: {error_msg}")
                            if "허용횟수" in error_msg:
                                print(f"\n[작업 중단] API 키의 일일 사용량을 소진했습니다.")
                                return
                            continue

                        person_info = result_json['peopleInfoResult']['peopleInfo']
                        save_single_person_to_csv(person_info, output_full_path)
                        collected_count += 1 # 성공 시에만 카운트 증가

                    except Exception as e:
                        print(f"  -> 정보 처리 중 오류 발생: {e}")
                    
                    time.sleep(0.1)

        except Exception as e:
            print(f"'{filename}' 파일 처리 중 오류가 발생했습니다: {e}")

    print("-" * 50)
    print("모든 목록을 확인했습니다.")
    if collected_count < MAX_TO_COLLECT:
        print(f"목표치({MAX_TO_COLLECT}개)에 도달하기 전에 모든 목록 확인이 완료되었습니다. (총 {collected_count}개 수집)")


if __name__ == "__main__":
    collect_specific_amount()

총 18개의 목록 CSV 파일을 발견했습니다. 3000개 추가 수집을 시작합니다.
--------------------------------------------------

📄 'kobis_people_list_1.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_10.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_11.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_12.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_13.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_14.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_15.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_16.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_17.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_18.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_2.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_3.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_4.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_5.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_6.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_7.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_8.csv' 파일 작업을 시작합니다...

📄 'kobis_people_list_9.csv' 파일 작업을 시작합니다...
--------------------------------------------------
모든 목록을 확인했습니다.
목표치(3000개)에 도달하기 전

In [ ]:
import requests
import json
import csv
import glob
import time
import os
import re

# --- 설정 ---
# 사용 가능한 모든 API 키를 리스트 형태로 입력합니다.
# 순서대로 사용하며, 하나를 다 쓰면 다음 키로 넘어갑니다.
API_KEYS = [
    "4dfdbf4abdc4df1cadf6f1d48f79ef80", 17분 씩
    "05fe695987e6b27c95631ba5bee9c601",
    "95fb114c7adc964e2a363a506cb32345",
    "d29200c5aa880538982113d33c889935",
    "3d9e9fb81650387f1b8f00b30e78b0f0",
    "0fedb3690ed2dbeb688fb9f77f9b7fdc"
    # 사용량이 초기화되면 여기에 다시 키를 추가할 수 있습니다.

]

# 상세정보를 저장할 폴더 경로
OUTPUT_FOLDER_PATH = r"C:\Users\yusse\OneDrive\바탕 화면\BAF-25-2-Marketing\배우추출_api\배우목록데이터"
# ------------

DETAIL_BASE_URL = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json"

def save_single_person_to_csv(person_data, full_path):
    """한 명의 상세정보를 CSV 파일로 저장하는 함수"""
    try:
        header = ['peopleCd', 'peopleNm', 'peopleNmEn', 'sex', 'repRoleNm', 'filmos', 'homepages']
        with open(full_path, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=header)
            writer.writeheader()
            person_data['filmos'] = json.dumps(person_data.get('filmos', []), ensure_ascii=False)
            person_data['homepages'] = json.dumps(person_data.get('homepages', []), ensure_ascii=False)
            writer.writerow(person_data)
        print(f"  -> ✅ 저장 완료: {os.path.basename(full_path)}")
    except Exception as e:
        print(f"  -> ❌ 저장 실패: {os.path.basename(full_path)} ({e})")

def get_and_save_details_fully_automated():
    """여러 API 키를 자동으로 교체하며 상세 정보를 수집하고, 모든 키 소진 시 작업을 종료합니다."""
    
    os.makedirs(OUTPUT_FOLDER_PATH, exist_ok=True)
    csv_files = glob.glob("kobis_people_list_*.csv")
    
    if not csv_files:
        print("[오류] 'kobis_people_list_*.csv' 파일을 찾을 수 없습니다.")
        return
        
    print(f"총 {len(csv_files)}개의 목록 CSV 파일을 발견했습니다. 상세 정보 개별 저장을 시작합니다.")
    print("-" * 50)
    
    current_key_index = 0
    total_processed_count = 0

    for filename in sorted(csv_files):
        print(f"\n📄 '{filename}' 파일 작업을 시작합니다...")
        
        try:
            with open(filename, 'r', encoding='utf-8-sig') as f:
                reader = csv.DictReader(f)
                
                for row in reader:
                    people_cd = row.get('peopleCd')
                    people_nm = row.get('peopleNm', '이름없음')
                    if not people_cd: continue

                    safe_people_nm = re.sub(r'[\\/*?:"<>|]', "", people_nm)
                    output_filename = f"{people_cd}_{safe_people_nm}.csv"
                    output_full_path = os.path.join(OUTPUT_FOLDER_PATH, output_filename)

                    if os.path.exists(output_full_path):
                        continue

                    total_processed_count += 1
                    print(f"\n[{total_processed_count}] {people_nm} ({people_cd}) 정보 수집 (현재 키: #{current_key_index + 1})")
                    
                    # --- '성공할 때까지 재시도' 및 '자동 키 교체' 로직 ---
                    while True:
                        params = {'key': API_KEYS[current_key_index], 'peopleCd': people_cd}
                        try:
                            response = requests.get(DETAIL_BASE_URL, params=params)
                            result_json = response.json()

                            if 'faultInfo' in result_json:
                                error_msg = result_json['faultInfo']['message']
                                print(f"  -> API 오류: {error_msg}")
                                
                                if "허용횟수" in error_msg:
                                    print(f"  -> API 키 #{current_key_index + 1}의 사용량을 모두 소진했습니다.")
                                    current_key_index += 1

                                    if current_key_index >= len(API_KEYS):
                                        print("\n[작업 중단] 오늘의 모든 API 키를 소진했습니다.")
                                        print("내일 다시 실행해주세요.")
                                        return
                                    
                                    print(f"  -> 다음 키(#{current_key_index + 1})로 자동 교체하여 재시도합니다...")
                                    time.sleep(1)
                                    continue # 같은 사람을 다음 키로 재시도
                                
                                break # 허용횟수 외 다른 오류는 다음 사람으로

                            person_info = result_json['peopleInfoResult']['peopleInfo']
                            save_single_person_to_csv(person_info, output_full_path)
                            break # 성공 시 루프 탈출

                        except Exception as e:
                            print(f"  -> 정보 처리 중 오류 발생: {e}")
                            break # 예외 발생 시 다음 사람으로
                    
                    time.sleep(0.1)

        except Exception as e:
            print(f"'{filename}' 파일 처리 중 오류가 발생했습니다: {e}")

    print("-" * 50)
    print("모든 파일의 상세 정보 저장 작업이 완료되었습니다.")

if __name__ == "__main__":
    get_and_save_details_fully_automated()